In [1]:
using ProgressMeter
using JLD
using Random
include("../alpcahus.jl")
include("../helperFunctions.jl")
include("../cluster.jl")

hungarianAlgorithm (generic function with 1 method)

In [2]:
pwd()

"/home/javier/Desktop/alpcahus/homoscedastic_kss"

In [2]:
D = 50 # ambient space dimension
d = 3 # subspace dimension
startingPoints = 6 # good points per subspace
v = 0.1 # noise variance good points
K = 2 # subspace count
varianceRatio = range( 1,150,5) # variance ratio for heatmaps
pointRatio = range(1,50,5) # point ratio between good bad data for heatmaps
trials = 100; # smoother results
qValue = 24;

In [4]:
seedVector1 = rand(Xoshiro(1234), 1:100000, trials*length(varianceRatio)*length(pointRatio));
seedVector2 = rand(Xoshiro(4321), 1:100000, trials*length(varianceRatio)*length(pointRatio));
seedVector3 = rand(Xoshiro(2143), 1:100000, trials*length(varianceRatio)*length(pointRatio));
seedVector4 = rand(Xoshiro(3412), 1:100000, trials*length(varianceRatio)*length(pointRatio));

In [5]:
ekssTensor = zeros(length(varianceRatio), length(pointRatio),trials)
alpcahTensor = zeros(length(varianceRatio), length(pointRatio),trials)
oracleTensor = zeros(length(varianceRatio), length(pointRatio),trials)

count = 1
@showprogress 1 "Running..." for i = 1:length(varianceRatio) # varianceRatio
     for j = 1:length(pointRatio) # prepend Threads.@threads for multi-threading
        for k = 1:trials
            # subspace generation
            U1 = generateSubspace(D, d; seed=seedVector1[count]) 
            U2 = generateSubspace(D, d; seed=seedVector2[count])
            # data generation
            totalPoints = round(Int,startingPoints*pointRatio[j])
            totalVariance = v*varianceRatio[i]
        
            X1 = generateDataHomo(U1, totalVariance, totalPoints ; seed=seedVector3[count])
            X2 = generateDataHomo(U2, totalVariance, totalPoints; seed=seedVector4[count])
            # complete data matrix
            X = hcat(X1,X2);
            # cluster assignments
            trueLabels = vec([1*ones(size(X1)[2],1); 2*ones(size(X2)[2],1)]);
            # ekss trial
            estimatedCluster = ALPCAHUS(X,K, [d,d]; B=1, q=qValue, T=100, subspaceMethod=:pca, finalStep=false)
            ekssTensor[i,j,k] = clusterError(trueLabels,estimatedCluster)
            # alpcahus trail
            estimatedCluster = ALPCAHUS(X,K, [d,d]; B=1, q=qValue, T=100, alpcahIter=100, subspaceMethod=:alpcah, finalStep=true, varfloor=1e-9, varianceMethod=:groupless) 
            alpcahTensor[i,j,k] = clusterError(trueLabels,estimatedCluster)
            # noisy oracle trial       
            noisyU1 = svd(X1).U[:,1:d]
            noisyU2 = svd(X2).U[:,1:d]
            estimatedCluster = clusterAssignment(X, [noisyU1, noisyU2], K)
            oracleTensor[i,j,k] = clusterError(trueLabels,estimatedCluster)
            # increase count
            count = count + 1
        end
    end
end

save("trialData.jld", "ekss", ekssTensor, "alpcah", alpcahTensor, "oracle", oracleTensor);

Running... 100%|█████████████████████████████████████████| Time: 1:10:40


In [3]:
trialData = load("trialData.jld")
ekssTensor = trialData["ekss"]
alpcahTensor = trialData["alpcah"]
oracleTensor = trialData["oracle"];

In [6]:
generateHeatmap3(pointRatio, varianceRatio, ekssTensor; plotType=:median, methodType="KSS", v=v, startingPoints=startingPoints)
savefig("kss_median.pdf")

"/home/javier/Desktop/alpcahus/homoscedastic_kss/kss_median.pdf"

In [7]:
generateHeatmap3(pointRatio, varianceRatio, alpcahTensor; plotType=:median, methodType="ALPCAHUS", v=v, startingPoints=startingPoints)
savefig("alpcahus_median.pdf")

"/home/javier/Desktop/alpcahus/homoscedastic_kss/alpcahus_median.pdf"

In [8]:
generateHeatmap3(pointRatio, varianceRatio, oracleTensor; plotType=:median, methodType="Noisy Oracle", v=v, startingPoints=startingPoints)
savefig("oracle_median.pdf")

"/home/javier/Desktop/alpcahus/homoscedastic_kss/oracle_median.pdf"

In [9]:
generateHeatmap3(pointRatio, varianceRatio, ekssTensor; plotType=:mean, methodType="KSS", v=v, startingPoints=startingPoints)
savefig("kss_mean.pdf")

"/home/javier/Desktop/alpcahus/homoscedastic_kss/kss_mean.pdf"

In [10]:
generateHeatmap3(pointRatio, varianceRatio, alpcahTensor; plotType=:mean, methodType="ALPCAHUS", v=v, startingPoints=startingPoints)
savefig("alpcahus_mean.pdf")

"/home/javier/Desktop/alpcahus/homoscedastic_kss/alpcahus_mean.pdf"

In [11]:
generateHeatmap3(pointRatio, varianceRatio, oracleTensor; plotType=:mean, methodType="Noisy Oracle", v=v, startingPoints=startingPoints)
savefig("oracle_mean.pdf")

"/home/javier/Desktop/alpcahus/homoscedastic_kss/oracle_mean.pdf"

In [12]:
generateHeatmap3(pointRatio, varianceRatio, ekssTensor; plotType=:variance, methodType="KSS", v=v, startingPoints=startingPoints)
savefig("kss_variance.pdf")

"/home/javier/Desktop/alpcahus/homoscedastic_kss/kss_variance.pdf"

In [13]:
generateHeatmap3(pointRatio, varianceRatio, alpcahTensor; plotType=:variance, methodType="ALPCAHUS", v=v, startingPoints=startingPoints)
savefig("alpcahus_variance.pdf")

"/home/javier/Desktop/alpcahus/homoscedastic_kss/alpcahus_variance.pdf"

In [14]:
generateHeatmap3(pointRatio, varianceRatio, oracleTensor; plotType=:variance, methodType="Noisy Oracle", v=v, startingPoints=startingPoints)
savefig("oracle_variance.pdf")

"/home/javier/Desktop/alpcahus/homoscedastic_kss/oracle_variance.pdf"